<a href="https://colab.research.google.com/github/kamilasouzac/Analise_Carrinho_de_Compras/blob/main/An%C3%A1lise_Carrinho_de_Compras_Parte_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RECAPITULANDO**

Estamos trabalhando com o dataset originalmente disponível no Kaggle para fazermos um estudo da cesta do carrinho de compras.

O dataset original pode ser oncnetrado em: https://www.kaggle.com/datasets/mittalvasu95/the-bread-basket

Ao final do projeto, queremos responder todas as seguintes perguntas:

1. Quais são os itens mais vendidos?
2. Quais items podem ser removidos do cardápio para dar espaço para novos pratos?
3. Qual período do dia a padaria mais vende?
4. O movimento no final de semana é maior do que durante a semana?
5. Existe alguma sazonalidade nas vendas?
6. Quais são os itens mais vendidos em conjunto?
7. Quais combos a padaria pode criar para estimular as vendas?

Na parte 1 (o notebook pode ser encontrado nesse mesmo repositório), tratamos nosso dataset, traduzindo os valores para facilitar o acesso do público brasileiro, verificamos se existiam dados nulos e transformamos o dtypes para melhor uso durante as análises.

Respondemos ainda as três primeiras perguntas, e preparamos nossa apresentação inicial para nosso cliente (o deck de slides também está disponível nesse repositório).

Na parte 2 responsemos as perguntas 4 e 5 e preparamos a apresentação dos resultados (todos os materiais podem ser concontrados nesse diretório).

Agora, vamos finalizar nosso estudo, respondendo as perguntas 6 e 7.

#**PREPANDANDO O AMBIENTE**

###Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

Na versão do pandas utilizada, eu estava recebendo a mensagem de erro abaixo:

 > /usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

O código abaixo foi a sulução para resolvermos o erro.

In [ ]:
pip install --upgrade ipykernel

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.2/154.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.1 MB/s eta 0:00:00
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.5.6
    Uninstalling ipykernel-5.5.6:
      Successfully uninstalled ipykernel-5.5.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.25.2 which is incompatible.


In [ ]:
print('Versão do Pandas utilizada:', pd.__version__)
print('Versão do Seaborn utilizada:', sns.__version__)

Versão do Pandas utilizada: 1.5.3
Versão do Seaborn utilizada: 0.12.2


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


###Parâmetros dos gráficos

In [ ]:
plt.rcParams['figure.figsize'] = (12, 6) #Tamanho do gráfico
sns.set_theme(style = 'whitegrid', palette = 'plasma')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#**IMPORTANDO O DATASET**

Vamos usar o DF já tratado na parte 1.

In [ ]:
df = pd.read_csv('df_tratado.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Transação,Item,Período_do_Dia,Período_da_Semana,Carimbo_data,Data,Dia,Mês,Ano,Hora,Hora_Completa
0,1,Pão,Manhã,Final_de_Semana,2016-10-30 09:58:00,2016-10-30,Domingo,Outubro,2016,9,09:58
1,2,Escandinavo,Manhã,Final_de_Semana,2016-10-30 10:05:00,2016-10-30,Domingo,Outubro,2016,10,10:05
2,2,Escandinavo,Manhã,Final_de_Semana,2016-10-30 10:05:00,2016-10-30,Domingo,Outubro,2016,10,10:05
3,3,Chocolate quente,Manhã,Final_de_Semana,2016-10-30 10:07:00,2016-10-30,Domingo,Outubro,2016,10,10:07
4,3,Geléia,Manhã,Final_de_Semana,2016-10-30 10:07:00,2016-10-30,Domingo,Outubro,2016,10,10:07


In [ ]:
df['Carimbo_data'] = pd.to_datetime(df['Carimbo_data'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df['Data'] = pd.to_datetime(df['Data'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## **6. Quais são os itens mais vendidos em conjunto?**

###**Regras de Associação**

Para responder nossas duas últimas perguntas, vamos usar uma Regra de Associação - um modelo de ML que nos ajuda a explorar as relações entre os itens da nossa amostra [1](https://medium.com/@bernardo.costa/uma-introdu%C3%A7%C3%A3o-ao-algoritmo-apriori-60b11293aa5a),[2](https://https://en.wikipedia.org/wiki/Association_rule_learning).

Aqui vamos usar o algoritimo Apriori. O moodelo trás quatro métricas importantes para a análise: suporte, lift, confiança e convicçaõ. Aqui uma definição super didática do [Vinicius dos Santos](https://www.computersciencemaster.com.br/como-funciona-o-algoritmo-apriori/):

> **SUPORTE**: "Este número é a popularidade do item no conjunto de dados estudado (dataset). Sendo assim esse número é definido pela quantidade de vezes que o item apareceu em uma transação, dividido pela quantidade de transações existentes no dataset". O suporte é calculado da seguinte maneira: supp(x) = (numero de transações que X aparece) / (numero total de transações)

> **CONFIANÇA:** "Se X for comprado, qual a chance de Y ser comprado?"

>"A confiança é um número que expressa a possibilidade de um item ser comprado quando outro item correlato é comprado. Por exemplo, qual a confiança que um cliente irá comprar um hambúrguer considerando que ele já comprou cebolas e batatas.   

>A confiança é calculada através da seguinte equação: conf(X→Y) = supp(X U Y) / supp(X)"


> **LIFT:** "Esta medida calcula também a possibilidade de um item ser comprado em relação a outro item. Porém, esta medida considera a popularidade de ambos os itens.

> lift(X→Y) = supp(X U Y) / supp(X) * supp(Y)

> Neste cálculo podemos analisar da seguinte forma: se o valor de lift(x→y) > 1 existe uma relação de compra entre estes dois itens. Já se o valor de lift(x→y) < 1 é muito provável que não exista uma relação clara expressa no dataset."

>**CONVICÇÃO:** "conv (x → y) = 1 – supp(y) / 1 – conf(x → y)

>Pode ser interpretado como a razão da frequência esperada que X ocorre sem Y (isto é, a frequência que a regra faz uma predição incorreta) se X e Y fossem independentes divididos pela frequência observada de predições incorretas."

Com as definições em mãos, vamos ao modelo. Primeiro, ajustaremos os dados com o *TransactionEncoder* e logo em seguida passaremos esse novo DF para o algoritimo.

In [ ]:
#Obtendo um "lista de listas" com os itens de cada transação

carrinhos = df.groupby('Transação')['Item'].apply(list).to_list()
carrinhos

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['Pão'],
 ['Escandinavo', 'Escandinavo'],
 ['Chocolate quente', 'Geléia', 'Biscoitos'],
 ['Muffin'],
 ['Café', 'Massa folhada', 'Pão'],
 ['Medialuna', 'Massa folhada', 'Muffin'],
 ['Medialuna', 'Massa folhada', 'Café', 'Chá'],
 ['Massa folhada', 'Pão'],
 ['Pão', 'Muffin'],
 ['Escandinavo', 'Medialuna'],
 ['Pão', 'Medialuna', 'Pão'],
 ['Geléia', 'Café', 'Hambúrguer no prato', 'Massa folhada', 'Chá'],
 ['Cesto', 'Pão', 'Café'],
 ['Pão', 'Medialuna', 'Massa folhada'],
 ['Água mineral', 'Escandinavo'],
 ['Pão', 'Medialuna', 'Café'],
 ['Chocolate quente'],
 ['Farm House'],
 ['Farm House', 'Pão'],
 ['Pão', 'Medialuna'],
 ['Café', 'Café', 'Medialuna', 'Pão'],
 ['Geléia'],
 ['Escandinavo', 'Muffin'],
 ['Pão'],
 ['Escandinavo'],
 ['Fudge'],
 ['Escandinavo'],
 ['Café', 'Pão'],
 ['Pão', 'Geléia'],
 ['Pão'],
 ['Cesto'],
 ['Escandinavo', 'Muffin'],
 ['Café'],
 ['Café', 'Muffin'],
 ['Muffin', 'Escandinavo'],
 ['Chá', 'Pão'],
 ['Café', 'Pão'],
 ['Pão', 'Chá'],
 ['Escandinavo'],
 ['Suco', 'Hambúrguer

In [ ]:
#Aplicando o encoding

te = TransactionEncoder()
te_ary = te.fit(carrinhos).transform(carrinhos)
te_ary

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
#Criando um novo DF a partir do encoding

df_carrinho = pd.DataFrame(te_ary, columns=te.columns_)
df_carrinho

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Bacon,Baguete,Bakewell,Bandeja de arte,Banquete vegano,Batatas fritas,...,Tigela Nic Pitt,Tigelas variadas,Torrada,Torta vegana de carne,Trufas,Vale-presente,Whey Protein,Xarope de gengibre,Água mineral,Óleo de chimichurri
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9460,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9461,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
9462,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9463,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Com nosso DF devidamente preparado, podemos obter a combinação de itens frequentes.

Precisamos passar qual o limite mínimo (*treshold*) que um conjunto aparece no universo amostral para poder ser incluido na lista de *itens frequentes*.

Por padrão, o Apriori considera esse limite como sendo 0.5. Mas testando esse limite, nenhum item/conjunto aparece. Após algumas tentativas, vamos considerar nosso limite mínimo 0.01 (ou seja, estamos pedindo para o algoritimo nos trazer quais itens e conjuntos aparecem em pelo menos 1% dos carrinhos).

In [ ]:
#Obtendo a lista de frequência de cada combinação

pd.set_option('display.precision', 2) #Mostra os resultados apenas com duas casas decimais

freq_items = apriori(df_carrinho, min_support= 0.01, use_colnames = True)
freq_items.sort_values('support', ascending = False).head(10).reset_index(drop=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.48,(Café)
1,0.33,(Pão)
2,0.14,(Chá)
3,0.10,(Bolo)
4,0.09,"(Pão, Café)"
5,0.09,(Massa folhada)
6,0.07,(Sanduíche)
7,0.06,(Medialuna)
8,0.06,(Chocolate quente)
9,0.05,"(Bolo, Café)"


**Observações:**

> O item mais frquente no nosso dataset é o Café, que aparece em 47.8% de todos os tickets. Logo em seguida no ranking, temos Pão, presente em 32.7% dos carrinhos, e o Chá, presente em 14% dos carrinhos.

> Quando olhamos para itens vendidos em conjunto, temos a combinação (Pão, Café) aparecendo em 9% dos carrinhos, e a combinação (Café, Bolo) aparecendo em 5% dos carrinhos.

## **7. Quais combos a padaria pode criar para estimular as vendas?**

Conhecidos a frequência dos itens mais vendidos, vamos para as regras de associação, e calcular o lift, confiança e convicção.

Aqui, queremos selecionar apenas as regras que tenham lift maior que 1. Ou seja, conjuntos onde exista uma relação de compra entre os itens.

In [ ]:
#Obtendo as medidas de confiança e lift

regras_associacao = association_rules(freq_items, metric = "lift", min_threshold = 1)
regras_associacao.sort_values('confidence', ascending = False, inplace = True)
regras_associacao[['antecedents', 'consequents', 'confidence', 'lift']].reset_index(drop=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,confidence,lift
0,(Torrada),(Café),0.70,1.47
1,(Brunch espanhol),(Café),0.60,1.25
2,(Medialuna),(Café),0.57,1.19
3,(Massa folhada),(Café),0.55,1.15
4,(Alfajores),(Café),0.54,1.13
5,(Suco),(Café),0.53,1.12
6,(Sanduíche),(Café),0.53,1.11
7,(Bolo),(Café),0.53,1.10
8,(Bolinho),(Café),0.52,1.09
9,(Biscoitos),(Café),0.52,1.08


In [ ]:
regras_associacao[['confidence', 'lift']].describe()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,confidence,lift
count,42.00,42.00
mean,0.25,1.26
std,0.21,0.27
min,0.02,1.02
25%,0.08,1.08
50%,0.13,1.13
75%,0.50,1.40
max,0.70,1.94


**Observações:**

>Temos uma lista com 42 recomendações de combinações de itens.

>Lift: a média de lift é apenas um pouco acima do treshold (1) para a medida.

>Confiança: a média de confiança da nossa lista é de cerca de 25%.

Café é o item mais vendido na padaria, portanto é natural que a algoritimo recomende o item quando outros são comprados. No entanto, estamos interessados nas associações menos óbvias, portanto desconsideraremos as sugestões em que café aparece como consequente.

Também queremos fazer um recorte em que a confiança seja superior a 10%.

In [ ]:
#Criando novo DF sem Café como possível item em consequente

regra_sem_cafe = regras_associacao[(regras_associacao['consequents'] != {'Café'}) &
                                   (regras_associacao['confidence'] > 0.1)]

#Visualizando o novo DF

regra_sem_cafe[['antecedents', 'consequents', 'confidence', 'lift']].reset_index(drop=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,confidence,lift
0,(Massa folhada),(Pão),0.34,1.03
1,(Bolo),(Chá),0.23,1.60
2,"(Chá, Café)",(Bolo),0.20,1.94
3,(Sanduíche),(Chá),0.20,1.40
4,(Chocolate quente),(Bolo),0.20,1.88
5,"(Café, Bolo)",(Chá),0.18,1.29
6,(Chá),(Bolo),0.17,1.60
7,(Massa folhada),"(Pão, Café)",0.13,1.44
8,"(Pão, Café)",(Massa folhada),0.12,1.44
9,(Café),(Bolo),0.11,1.10


**Observações:**

> Chegamos a uma lista final com 13 combos que a padaria pode criar para estimular as vendas.